In [1]:
# 필요한 패키지 임포트
import random
import torch
import pandas as pd
import numpy as np

# 랜덤시드 고정
seed = 1
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [2]:
# 데이터 로드
train = pd.read_csv('../input/2022-ai-midterm-p1/train.csv')
test = pd.read_csv('../input/2022-ai-midterm-p1/test.csv')
submit = pd.read_csv('../input/2022-ai-midterm-p1/submit_sample.csv')

In [3]:
# 필요없는 column drop 및 정답 라벨 분리
y = train.drop('index', axis=1).label
test = test.drop('index', axis=1)
x = train.drop(['index', 'label'], axis=1)

In [4]:
# torch tensor로 옮기기
x_tensor = torch.FloatTensor(x.to_numpy()).cuda()
y_tensor = torch.LongTensor(y).cuda()
test_tensor = torch.FloatTensor(test.to_numpy()).cuda()

In [5]:
# 모델 설계
layer1 = torch.nn.Linear(x_tensor.shape[1], 512).cuda()
layer2 = torch.nn.Linear(512, 512).cuda()
layer3 = torch.nn.Linear(512, 1024).cuda()
layer4 = torch.nn.Linear(1024, 10).cuda()

# 활성화 함수
relu = torch.nn.ReLU()
# Dropout
dropout = torch.nn.Dropout(p=0.3)

# 레이어 init
torch.nn.init.xavier_normal_(layer1.weight)
torch.nn.init.xavier_normal_(layer2.weight)
torch.nn.init.xavier_normal_(layer3.weight)
torch.nn.init.xavier_normal_(layer4.weight)

# 레이어 연결
model = torch.nn.Sequential(
    layer1, relu, dropout,
    layer2, relu, dropout,
    layer3, relu, dropout,    
    layer4
).cuda()

#loss 함수 설정
loss = torch.nn.CrossEntropyLoss().cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

In [6]:
# 모델 학습
epochs = 12345
for epoch in range(epochs):
    optimizer.zero_grad()
    cost = loss(model(x_tensor), y_tensor)
    cost.backward()
    optimizer.step()
    
    if epoch % 1000 == 0:
        print(epoch, cost.item())

0 4.346856594085693
1000 0.009221653454005718
2000 0.0008067984599620104
3000 0.00024503047461621463
4000 0.00014635061961598694
5000 4.2977902921848e-05
6000 1.2174992662039585e-05
7000 0.00035509539884515107
8000 1.3445119293464813e-05
9000 0.00010253094660583884
10000 4.097610144526698e-06
11000 1.8893548485721112e-06
12000 1.949435863934923e-05


In [7]:
# 정답 도출
predict = model(test_tensor).cpu().argmax(dim=1)
submit.label = predict.detach()
submit.to_csv('submission.csv', index=False)